In [114]:
from PIL import Image
from transformers.image_utils import load_image
img = load_image("data/019/IMG_20240220_174554771.jpg")
img2 = load_image("v379-sasi-11-terrazzo-g.jpg")

In [115]:
import numpy as np
from segment_anything import SamPredictor, sam_model_registry
sam = sam_model_registry["vit_b"](checkpoint="/Users/timothee/Downloads/sam_vit_b_01ec64.pth")
predictor = SamPredictor(sam)
predictor.set_image(np.array(img))
masks, _, _ = predictor.predict()

In [83]:
def get_segmentation(image: Image, predictor: SamPredictor):
    predictor.set_image(np.array(image))
    mask, _, _ = predictor.predict()

    return mask

In [119]:
masked = np.where(~mask[..., None], img_array, 0)

In [120]:
masked

array([[[118, 111,  95],
        [109, 102,  86],
        [107, 101,  85],
        ...,
        [226, 206, 231],
        [224, 204, 229],
        [222, 202, 227]],

       [[114, 107,  91],
        [106,  99,  83],
        [104,  98,  82],
        ...,
        [223, 203, 228],
        [222, 202, 227],
        [221, 201, 226]],

       [[108, 100,  87],
        [105,  97,  84],
        [104,  98,  84],
        ...,
        [223, 203, 228],
        [222, 202, 227],
        [222, 202, 227]],

       ...,

       [[ 31,  26,  46],
        [ 28,  23,  43],
        [ 32,  27,  47],
        ...,
        [ 81,  40,  48],
        [ 76,  35,  43],
        [ 71,  30,  38]],

       [[ 30,  25,  47],
        [ 30,  25,  47],
        [ 33,  28,  50],
        ...,
        [ 85,  40,  47],
        [ 80,  35,  42],
        [ 70,  25,  32]],

       [[ 30,  25,  47],
        [ 32,  27,  49],
        [ 34,  29,  51],
        ...,
        [ 77,  32,  39],
        [ 79,  34,  41],
        [ 76,  31,  38]]

In [116]:
mask = masks[0]  # shape (H, W)
img_array = np.array(img)

# Invert mask if necessary to keep the object and remove background
masked = np.where(~mask[..., None], img_array, 0)  # keep object pixels

Image.fromarray(masked.astype(np.uint8)).show()

In [121]:
mask = masks[0]
img_array = np.array(img)

# 1. Get bounding box
coords = np.argwhere(mask)
print(coords)
ymin, xmin = coords.min(axis=0)
ymax, xmax = coords.max(axis=0) + 1

# 2. Crop
cropped_img = img_array[ymin:ymax, xmin:xmax]
cropped_mask = mask[ymin:ymax, xmin:xmax]

# 3. Apply mask inside crop
final = np.where(~cropped_mask[..., None], cropped_img, 0)

# 4. Resize to classifier input size (e.g. 224x224)
from PIL import Image
final_pil = Image.fromarray(final.astype(np.uint8)).resize((224, 224))

[]


ValueError: zero-size array to reduction operation minimum which has no identity

In [94]:
final_pil.show()

In [122]:
import numpy as np
from PIL import Image

def replace_background(img: Image.Image, mask: np.ndarray, new_bg: Image.Image, size: int = 224) -> Image.Image:
    """
    Crops an object based on its mask and replaces the background with another image.
    
    Args:
        img (PIL.Image): Original input image.
        mask (np.ndarray): Boolean mask (H, W), True where object is.
        new_bg (PIL.Image): Replacement background image.
        size (int): Output image size (size x size).
        
    Returns:
        PIL.Image: Object on new background, resized.
    """
    img_array = np.array(img)
    bg_array = np.array(new_bg.resize(img.size))  # match background to original size
    
    # Ensure background has 3 channels
    if bg_array.ndim == 2:  # grayscale → RGB
        bg_array = np.stack([bg_array]*3, axis=-1)
    
    # Apply mask: object from original, background from new_bg
    composite = np.where(~mask[..., None], img_array, bg_array)
    
    # Crop tightly around the object
    """ coords = np.argwhere(mask)
    ymin, xmin = coords.min(axis=0)
    ymax, xmax = coords.max(axis=0) + 1
    composite_crop = composite[ymin:ymax, xmin:xmax] """
    
    # Resize to classifier input size
    final_pil = Image.fromarray(composite.astype(np.uint8)).resize((size, size))
    
    return final_pil

replace_background(img, mask, img2).show()

In [58]:
import pandas as pd
import os
import glob	

path = '/Users/timothee/Downloads/unsplash-research-dataset-lite-latest/photos.csv000'
df = pd.read_csv(path, sep='\t', header=0)


In [80]:
replace_background(img, mask, load_image(df["photo_image_url"][0])).save("./cool.jpg")

In [102]:
import random
random.randint(0, len(df)-1)

17497

In [125]:
from pathlib import Path

base_path = Path("data")
new_source_path = Path("augmented_data16")
image_num_multiplier = 4
base_image_num = 4

for cap_dir in base_path.iterdir():
    label = cap_dir.as_posix().split("/")[-1]
    subpath = new_source_path.joinpath(label)
    if cap_dir.is_dir() and (not subpath.exists() or sum([1 for _ in subpath.iterdir()]) < image_num_multiplier*base_image_num):
        new_source_path.joinpath(label).mkdir(exist_ok=True)
        for image_path in cap_dir.iterdir():
            print(image_path)
            image_name = image_path.as_posix().split("/")[-1]
            image = load_image(image_path.as_posix())
            mask = get_segmentation(np.array(image), predictor)[0]
            for n in range(image_num_multiplier):
                seed = random.randint(0, len(df)-1)
                background_image = load_image(df["photo_image_url"][seed])
                augmented_image = replace_background(image, mask, background_image)
                augmented_image.save(subpath.joinpath('-'.join([str(n), image_name])).as_posix())

data/102/IMG_20240220_214944675.jpg
data/102/IMG_20240220_225319499.jpg
data/102/IMG_20240220_174237969.jpg
data/102/IMG_20240220_171422648.jpg
data/024/IMG_20240220_224257510.jpg
data/024/IMG_20240220_171311334.jpg
data/024/IMG_20240220_220421848.jpg
data/024/IMG_20240220_175440002.jpg
data/023/IMG_20240220_225028497.jpg
data/023/IMG_20240220_215723601.jpg
data/023/IMG_20240220_175621292.jpg
data/023/IMG_20240220_171305531.jpg
data/015/IMG_20240220_175042472.jpg
data/015/IMG_20240220_220808999.jpg
data/015/IMG_20240220_171138085.jpg
data/015/IMG_20240220_224309610.jpg
data/012/IMG_20240220_224514788.jpg
data/012/IMG_20240220_174428896.jpg
data/012/IMG_20240220_220635060.jpg
data/012/IMG_20240220_171113946.jpg
data/079/IMG_20240220_224416681.jpg
data/079/IMG_20240220_172711051.jpg
data/079/IMG_20240220_174739839.jpg
data/079/IMG_20240220_221224954.jpg
data/046/IMG_20240220_171819159.jpg
data/046/IMG_20240220_174307607.jpg
data/046/IMG_20240220_224406695.jpg
data/046/IMG_20240220_221312